<a href="https://colab.research.google.com/github/masoud-moghini/computer-vision/blob/main/chapter_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
from pathlib import Path
import numpy as np

In [ ]:
!wget -O train-annotations-object-segmentation.csv -q https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv
!wget -O classes.csv -q https://raw.githubusercontent.com/openimages/dataset/master/dict.csv

In [40]:
cd ..

/content


In [5]:
# Read the original CSV file
df = pd.read_csv('classes.csv')

# Select the first 100 rows
first_100_rows = df.head(100)

# Save the first 100 rows to a new CSV file
first_100_rows.to_csv('classes_100.csv', index=False)

print("First 100 rows saved to 'classes_100.csv'")

First 100 rows saved to 'classes_100.csv'


In [6]:
required_classes = 'person,dog,bird,car,elephant,football,\
jug,laptop,mushroom,pizza,rocket,shirt,traffic sign,\
Watermelon,Zebra'
required_classes = required_classes.lower().split(',')
classes = pd.read_csv('classes.csv')
classes.columns = ['class','class_name']
classes = classes[classes['class_name'].isin(required_classes)]

In [ ]:
classes

,class,class_name
42,/m/01226z,football
223,/m/015p6,bird
600,/m/01c648,laptop
752,/m/01g317,person
1021,/m/01mqdt,traffic sign
1044,/m/01n4qj,shirt
2818,/m/02vx4,football
4674,/m/052sf,mushroom
5180,/m/0663v,pizza
5915,/m/0898b,zebra


In [43]:
df = pd.read_csv('train-annotations-object-segmentation.csv')
data = pd.merge(df,classes,left_on='LabelName',right_on='class',how='inner')

In [44]:
subset_data = data.groupby('class_name').agg({'ImageID':lambda x: list(x)[:500]})
flattened_data = subset_data.explode('ImageID').reset_index()
subset_data =  data[data['ImageID'].map(lambda x :x in flattened_data['ImageID'].tolist())]
subset_mask = subset_data['MaskPath'].tolist()

##Download google open images masks dataset

In [ ]:
!mkdir -p masks
for c in tqdm('0123456789abcdef'):
  !wget -q https://storage.googleapis.com/openimages/v5/train-masks/train-masks-{c}.zip
  !unzip -q train-masks-{c}.zip -d tmp_masks
  !rm train-masks-{c}.zip
  tmp_masks = glob('tmp_masks/*')
  items = [(m,Path(m).name) for m in tmp_masks]
  items = [(i,j) for (i,j) in items if j in subset_masks]
  for i,j in items:
    os.rename(i, f'masks/{j}')
  !rm -rf tmp_masks

100%|██████████| 16/16 [20:14<00:00, 75.92s/it]


In [ ]:
!pip install openimages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
masks = glob('masks/*')
masks = [Path(mask).name for mask in masks]
subset_data=subset_data[subset_data['MaskPath'].map(lambda x: x in masks)]
subset_imageIds = subset_data['ImageID'].tolist()
from openimages.download import _download_images_by_id
!mkdir images
_download_images_by_id(subset_imageIds, 'train', './images/')

100%|██████████| 14090/14090 [04:15<00:00, 55.08it/s]


# compress and move data to drive

In [ ]:
import zipfile
files = glob('images/*') + glob('masks/*') + ['train-annotations-object-segmentation.csv', 'classes.csv']
with zipfile.ZipFile('data.zip','w') as zipme:
  for file in tqdm(files):
    zipme.write(file,compress_type=zipfile.ZIP_DEFLATED)

100%|██████████| 20005/20005 [02:58<00:00, 111.97it/s]


In [ ]:
mv data.zip /content/drive/MyDrive/computer-vision/chapter10/

In [ ]:
!unzip /content/drive/MyDrive/computer-vision/chapter10/data.zip

In [20]:
!git clone https://github.com/sizhky/pycococreator.git

fatal: destination path 'pycococreator' already exists and is not an empty directory.


In [28]:
cd pycococreator/

/content/pycococreator


In [30]:
pip install .

Processing /content/pycococreator
  Preparing metadata (setup.py) ... done
  Created wheel for pycococreatortools: filename=pycococreatortools-0.2.0-py3-none-any.whl size=7182 sha256=92f10b93e67970821340cec2f61cd5023977c54f30f7efbf6b7d4f4029f3029d
  Stored in directory: /tmp/pip-ephem-wheel-cache-35fdpoj_/wheels/8a/fe/69/ea758123996b682641265c8d060315904b3622264189253da4
Successfully built pycococreatortools


In [45]:
import datetime
INFO = {
  "description": "MyData2020",
  "url": "None",
  "version": "1.0",
  "year": 2020,
  "contributor": "sizhky",
  "date_created": datetime.datetime.utcnow().isoformat(' ')
}
LICENSES = [
  {
    "id": 1,
    "name": "MIT"
  }
]
CATEGORIES = [{'id': id+1, 'name': name.replace('/',''),
  'supercategory': 'none'} \
  for id,(_,(name, clss_name)) in \
  enumerate(classes.iterrows())]

In [46]:
from pycococreatortools import pycococreatortools
from os import listdir
from os.path import isfile,join
from PIL import Image

coco_output = {
  "info": INFO,
  "licenses": LICENSES,
  "categories": CATEGORIES,
  "images": [],
  "annotations": []
}


ROOT_DIR = "train"
IMAGE_DIR, ANNOTATION_DIR = 'train/images/', 'train/masks/'
image_files = [f for f in listdir(IMAGE_DIR) if isfile(join(IMAGE_DIR, f))]
annotation_files = [f for f in listdir(ANNOTATION_DIR) if isfile(join(ANNOTATION_DIR, f))]


image_id = 1
# go through each image
for image_filename in tqdm(image_files):
  image = Image.open(IMAGE_DIR + '/' + image_filename)
  image_info = pycococreatortools.create_image_info(image_id, \
  os.path.basename(image_filename), image.size)
  coco_output["images"].append(image_info)
  image_id = image_id + 1

100%|██████████| 5913/5913 [00:00<00:00, 9596.17it/s] 


In [58]:
segmentation_id = 1
for annotation_filename in tqdm(annotation_files):
  image_id = [f for f in coco_output['images'] if  Path(f['file_name']).stem == \
              annotation_filename.split('_')[0]][0]['id']
  class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename]\
   [0]
  category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
  binary_mask = np.asarray(Image.open(f'{ANNOTATION_DIR}/{annotation_filename}')\
                           .convert('1')).astype(np.uint8)
  annotation_info = pycococreatortools.create_annotation_info(
  segmentation_id, image_id,category_info,binary_mask, image.size, tolerance=2)
  if annotation_info is not None:
    coco_output["annotations"].append(annotation_info)
    segmentation_id = segmentation_id + 1

100%|██████████| 14090/14090 [14:11<00:00, 16.55it/s]


In [59]:
coco_output['categories'] = [{'id': id+1,'name':clss_name, 'supercategory': 'none'} for \
                    id,(_,(name, clss_name)) in enumerate(classes.iterrows())]
import json
with open('images.json', 'w') as output_json_file:
  json.dump(coco_output, output_json_file)

In [60]:
mv images.json drive/MyDrive/computer-vision/chapter10/

In [64]:
%cd /content/
# install detectron2:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
%pip install git+https://github.com/facebookresearch/fvcore.git
%cd /content/

/content
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/iopath-0.1.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/detectron2-0.6-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/hydra_core-1.4.0.dev1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/black-25.1.0-py3.11-linux-x86_64.egg is deprecated

In [66]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/iopath-0.1.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/detectron2-0.6-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/black-25.1.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/mypy_extensions-1.1.0-py3.11.egg is deprecated. pip 24.

In [2]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/images.json", "/content/train")

In [9]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODELS_WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")


#weights
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)